Test 1 - Using VGG with last embedded layer and Random Forest at the end

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [2]:
import tensorflow as tf
import numpy as np

In [3]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [9]:
tf.__version__

'2.10.1'

In [46]:
# Analyze + process data
#Filter by number of entries
train_dir = "C:/Users/malik/Desktop/ASL-Aly/ASL-words-translator/IMAGE_vision/train_data"

saved_classes = []

for folder in os.listdir(train_dir):
    if len(os.listdir(os.path.join(train_dir, folder))) >= 15:
        saved_classes.append(folder)
        
    # for file in os.walk(os.path.join(train_dir, dirnames)):
    #     print(file)
(saved_classes)

['afraid',
 'afternoon',
 'again',
 'all',
 'apple',
 'aunt',
 'bad',
 'bathroom',
 'beautiful',
 'black',
 'blue',
 'book',
 'born',
 'boy',
 'boyfriend',
 'bread',
 'brother',
 'brown',
 'bus',
 'bye',
 'cat',
 'cheese',
 'chicken',
 'children',
 'church',
 'class',
 'clothes',
 'coffee',
 'college',
 'color',
 'computer',
 'cook',
 'cousin',
 'cup',
 'dance',
 'daughter',
 'day',
 'deaf',
 'die',
 'doctor',
 'dog',
 'door',
 'draw',
 'drink',
 'eat',
 'egg',
 'english',
 'family',
 'father',
 'fine',
 'finish',
 'fish',
 'forget',
 'france',
 'friend',
 'friendly',
 'girl',
 'gold',
 'good',
 'grandfather',
 'grandmother',
 'green',
 'hair',
 'happy',
 'hat',
 'have',
 'hearing',
 'hello',
 'help',
 'here',
 'home',
 'horse',
 'how',
 'how_many',
 'hungry',
 'hurt',
 'husband',
 'jacket',
 'kitchen',
 'know',
 'late',
 'learn',
 'light',
 'like',
 'live',
 'lost',
 'love',
 'lunch',
 'mad',
 'man',
 'marry',
 'me',
 'meet',
 'milk',
 'morning',
 'mother',
 'my',
 'name',
 'nephew',


In [62]:
# Filter by frames
new_saved = []
for label in saved_classes:
    dir = os.path.join(train_dir, label)
    for file in os.listdir(dir):
        arr = np.load(os.path.join(train_dir, label, file))
        if len(arr) < 2:
            break
        elif os.listdir(dir)[-1] == file:
            new_saved.append(label)

print(len(new_saved))

24


In [64]:
# Data with at least 15 entries, each having at least 2 frames
new_saved 

['apple',
 'cheese',
 'chicken',
 'church',
 'coffee',
 'college',
 'cook',
 'daughter',
 'dog',
 'door',
 'friendly',
 'grandmother',
 'home',
 'hungry',
 'milk',
 'not know',
 'shoes',
 'thursday',
 'time',
 'tomorrow',
 'uncle',
 'vacation',
 'woman',
 'your']

In [91]:
# Add each video into one collective Numpy Array, containing frames (3D objects)

X = []
Y = []
for label in new_saved:
    print(label)
    dir = os.path.join(train_dir, label)
    for file in os.listdir(dir):
        arr = np.load(os.path.join(train_dir, label, file))
        for frame in arr:
            if len(X) == 0:
                X = np.array([frame])
                Y = np.array([new_saved.index(label)])
            else:
                X = np.append(X, np.array([frame]), axis=0)
                Y = np.append(Y, np.array([new_saved.index(label)]), axis=0)

print(X.shape, Y.shape)

apple
cheese
chicken
church
coffee
college
cook
daughter
dog
door
friendly
grandmother
home
hungry
milk
not know
shoes
thursday
time
tomorrow
uncle
vacation
woman
your
(1404, 256, 256, 3) (1404,)


In [3]:
np.save("C:/Users/malik/Desktop/ASL-Aly/ASL-words-translator/IMAGE_vision/X", np.asarray(X, dtype=np.float32), allow_pickle=True, fix_imports=True)
np.save("C:/Users/malik/Desktop/ASL-Aly/ASL-words-translator/IMAGE_vision/Y", np.asarray(Y, dtype=np.float32), allow_pickle=True, fix_imports=True)

NameError: name 'X' is not defined

In [93]:
new_saved = ['apple',
 'cheese',
 'chicken',
 'church',
 'coffee',
 'college',
 'cook',
 'daughter',
 'dog',
 'door',
 'friendly',
 'grandmother',
 'home',
 'hungry',
 'milk',
 'not know',
 'shoes',
 'thursday',
 'time',
 'tomorrow',
 'uncle',
 'vacation',
 'woman',
 'your']

In [4]:
X = np.load("C:/Users/malik/Desktop/ASL-Aly/ASL-words-translator/IMAGE_vision/X.npy")
Y = np.load("C:/Users/malik/Desktop/ASL-Aly/ASL-words-translator/IMAGE_vision/Y.npy")
X.shape, Y.shape

((1404, 256, 256, 3), (1404,))

In [5]:
import sklearn
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)


In [6]:
from tensorflow.keras.utils import Sequence
import numpy as np   

class DataGenerator(Sequence):
    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        return batch_x, batch_y

train_gen = DataGenerator(X_train, y_train, 32)
test_gen = DataGenerator(X_test, y_test, 32)

In [7]:
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16

vgg_model = VGG16(weights='imagenet', input_shape=(256, 256, 3), include_top=False)
vgg_model.trainable = False
vgg_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 256, 256, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 256, 256, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 128, 128, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 128, 128, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 128, 128, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 64, 64, 128)       0     

In [8]:
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model

x = vgg_model.get_layer('block5_pool').output
x = Flatten(name='flatten')(x)
# fc1 = Dense(4096, activation='relu')(x)
# fc2 = Dense(4096, activation='relu')(fc1)
output = Dense(24, activation='softmax')(x)

vgg_model = Model(vgg_model.input, output)

In [9]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

In [9]:
gpus = tf.config.experimental.list_physical_devices("GPU")
if gpus:
  # Restrict TensorFlow to only allocate 1GB of memory on the first GPU
  try:
    tf.config.experimental.set_virtual_device_configuration(gpus[0],
    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=2048)])
    logical_gpus = tf.config.experimental.list_logical_devices("GPU")
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
  # Virtual devices must be set before GPUs have been initialized
    print(e)

Virtual devices cannot be modified after being initialized


In [10]:
gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.333) 
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True, gpu_options=gpu_options))

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: NVIDIA GeForce GTX 1660 SUPER, pci bus id: 0000:06:00.0, compute capability: 7.5



In [11]:
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    "C:/Users/malik/Desktop/ASL-Aly/ASL-words-translator/training_models/weights.{epoch:02d}-{val_loss:.2f}",
    monitor='val_loss',
    verbose=0,
    save_best_only=True,
    save_weights_only=False,
    mode='auto',
    save_freq='epoch',
)

In [12]:
vgg_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
vgg_model.fit(X_train, y_train, epochs=50, batch_size=10, validation_split=0.2, callbacks=[checkpoint])

TypeError: Invalid keyword argument(s) in `compile()`: ({'callbacks'},). Valid keyword arguments include "cloning", "experimental_run_tf_function", "distribute", "target_tensors", or "sample_weight_mode".

In [ ]:
vgg_model.layers[-2].output

In [ ]:
from keras import backend as K
import tensorflow as tf

get_fc1_output = K.function([vgg_model.layers[0].input],[vgg_model.layers[-2].output])

test_sample = tf.random.uniform(shape=(1,224,224,3), minval=0, maxval=1)
# test_sample.shape
layer_output = get_fc1_output([test_sample])[0]
layer_output